<a href="https://colab.research.google.com/github/AhnYeonghoo/Plastic-self-classification-system-using-deep-learning/blob/Machine-Learning/data_preprocessing1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%이 부분은 구글 드라이브 마운트 하는 부분이라 너가 로컬에서 할거면 바꿔서 해야해
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import glob
import zipfile
import shutil

In [ ]:
# %%이건 갯수 맞춰보는거. 파일의 폴더 경로에 따라 바꿔서 해야해.

# 두 개의 폴더 경로 설정
LabeledData_path = '/content/drive/MyDrive/01-1.정식개방데이터/Validation/A1/Labeled_Data_A1/A1'
SourceData_path = '/content/drive/MyDrive/01-1.정식개방데이터/Validation/A1/Source_Data_A1/A1'

# 폴더 내 파일 개수 세기
LabeledData_count = len([name for name in os.listdir(LabeledData_path) if os.path.isfile(os.path.join(LabeledData_path, name))])
SourceData_count = len([name for name in os.listdir(SourceData_path) if os.path.isfile(os.path.join(SourceData_path, name))])

# 결과 출력
print(f"라벨링데이터의 파일 개수: {LabeledData_count}")
print(f"원천데이터의 파일 개수: {SourceData_count}")

In [ ]:
import json
###########################################################
# %% 이건 아까 말했던 갯수 손실난 부분 갯수 맞추는건데 너는 로컬에서 압축해제 하니까 손실 없을 수도 있어. 근데 손실 없더라도 혹시 모르니까 이 부분 꼭 돌려봐줘.
###########################################################


# 이미지와 라벨 파일 경로 설정 (구글 드라이브 내 경로로 설정)
image_dir = "/content/drive/MyDrive/01-1.정식개방데이터/Validation/A1/Source_Data_A1/A1"
label_dir = "/content/drive/MyDrive/01-1.정식개방데이터/Validation/A1/Labeled_Data_A1/A1"

# 이미지와 라벨 파일 리스트 얻기
image_files = set([os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith('.jpg')])
label_files = set([os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith('.json')])

# 이미지와 라벨 모두 있는 파일 리스트 찾기
valid_files = image_files.intersection(label_files)

# 이미지나 라벨만 있는 파일 리스트
only_image_files = image_files - valid_files
only_label_files = label_files - valid_files

# 이미지나 라벨 중 하나만 있는 파일 삭제 (필요시)
for file in only_image_files:
    os.remove(os.path.join(image_dir, file + '.jpg'))

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# %% 이게 제일 중요!!!!!! 이거랑 똑같은 코드가 뒤에 전처리 한 후에 또 쓰는거거든 근데 전처리를 하기 전에 라벨 데이터가 json파일이고 전처리 후에는 txt 파일이야. 그래서 이 주석 밑에 코드 돌릴때 json인지 txt인지 잘 보고 돌려야해.
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
for file in only_label_files:
    os.remove(os.path.join(label_dir, file + '.json'))


print(f"총 {len(valid_files)}개의 유효한 이미지와 라벨 파일이 있습니다.")


In [ ]:
import os
import cv2
import json
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
#######################
# 데이터 전처리해서 output_dir에 저장하는거. 이제 여기서는 이미지 데이터, 라벨 데이터가 하나의 폴더에 합쳐질거야.
#######################
# 경로 설정
label_dir = '/content/drive/MyDrive/01-1.정식개방데이터/Validation/A1/Labeled_Data_A1/A1'
image_dir = '/content/drive/MyDrive/01-1.정식개방데이터/Validation/A1/Source_Data_A1/A1'
output_dir = '/content/drive/MyDrive/01-1.정식개방데이터/Validation/A1/Output_Data_A1'  # 이미지와 라벨 파일 저장 경로

os.makedirs(output_dir, exist_ok=True)

# YOLO에서 사용할 고정된 크기
TARGET_SIZE = 640

# 클래스 매핑 (클래스 이름을 숫자 ID로 변환)
class_mapping = {
    "c_5_02": 0,
    "c_6": 1,
    "c_5_01_01": 2,
    "c_5_02_01": 3,
    "c_6_01": 4,
    "c_5_01": 5
}

# 비율 유지하면서 이미지 축소 후 패딩 추가 함수
def resize_with_padding(image, target_size):
    h, w = image.shape[:2]
    scale = target_size / max(w, h)
    new_w = int(w * scale)
    new_h = int(h * scale)

    resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # 패딩 추가
    delta_w = target_size - new_w
    delta_h = target_size - new_h
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    padded_image = cv2.copyMakeBorder(resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    return padded_image, scale, left, top

# 바운딩 박스 좌표 변환 함수
def convert_bbox(bbox, scale, left_pad, top_pad):
    x_min = bbox['x'] * scale + left_pad
    y_min = bbox['y'] * scale + top_pad
    width = bbox['width'] * scale
    height = bbox['height'] * scale

    # YOLO 형식으로 변환 (x_center, y_center, width, height)
    x_center = (x_min + width / 2) / TARGET_SIZE
    y_center = (y_min + height / 2) / TARGET_SIZE
    width /= TARGET_SIZE
    height /= TARGET_SIZE

    return x_center, y_center, width, height

# 이미지와 JSON 파일을 처리하는 함수
def process_image_and_label(image_path, label_path, output_dir):
    # 이미지 읽기
    image = cv2.imread(image_path)

    # 이미지 리사이즈 (비율 유지 + 패딩)
    padded_image, scale, left_pad, top_pad = resize_with_padding(image, TARGET_SIZE)

    # 출력 이미지 저장
    image_name = os.path.basename(image_path)
    output_image_path = os.path.join(output_dir, image_name)
    cv2.imwrite(output_image_path, padded_image)

    # JSON 파일 읽기
    with open(label_path, 'r') as f:
        data = json.load(f)

    # YOLO 형식의 라벨 파일 생성
    label_name = os.path.splitext(image_name)[0] + '.txt'
    output_label_path = os.path.join(output_dir, label_name)

    with open(output_label_path, 'w') as label_file:
        for obj in data['objects']:
            class_name = obj['class_name']  # 클래스 이름
            bbox = obj['annotation']['coord']  # 바운딩 박스 좌표

            # 바운딩 박스 좌표 변환
            x_center, y_center, width, height = convert_bbox(bbox, scale, left_pad, top_pad)

            # 클래스 ID 가져오기
            class_id = class_mapping.get(class_name, -1)  # class_mapping에서 클래스 ID 찾기, 없으면 -1 반환
            if class_id == -1:
                print(f"Warning: 클래스 '{class_name}'에 대한 매핑을 찾을 수 없습니다.")
                continue

            # YOLO 형식으로 라벨 저장
            label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# 병렬 처리 함수
def process_parallel():
    with ThreadPoolExecutor() as executor:
        futures = []
        for label_file in os.listdir(label_dir):
            if label_file.endswith('.json'):
                image_file = label_file.replace('.json', '.jpg')
                image_path = os.path.join(image_dir, image_file)
                label_path = os.path.join(label_dir, label_file)

                futures.append(executor.submit(process_image_and_label, image_path, label_path, output_dir))

        # 완료된 작업 확인
        for future in as_completed(futures):
            try:
                future.result()  # 결과를 받아서 예외를 처리할 수 있음
            except Exception as e:
                print(f"Error: {e}")

# 병렬 처리 실행
process_parallel()

print("모든 이미지와 라벨이 병렬 처리되었습니다!")


In [ ]:
import shutil

# 새로운 출력 경로 설정 (유효한 데이터만 저장할 경로)
new_output_dir = '/content/drive/MyDrive/yolo_training_A1/filtered_val'
output_dir = '/content/drive/MyDrive/yolo_training_A1/val'

os.makedirs(new_output_dir, exist_ok=True)

# 빈 라벨 및 해당 이미지 파일의 수를 확인하고, 유효한 데이터만 복사하는 함수
def filter_and_copy_valid_data(output_dir, new_output_dir):
    invalid_pairs_count = 0

    for file_name in os.listdir(output_dir):
        if file_name.endswith('.txt'):
            file_path = os.path.join(output_dir, file_name)
            if os.path.getsize(file_path) == 0:  # 파일 크기가 0이면 쓸모없는 데이터로 간주
                # 같은 이름의 이미지 파일 경로
                image_file = file_name.replace('.txt', '.jpg')
                image_path = os.path.join(output_dir, image_file)

                # 삭제할 데이터 쌍 개수 세기
                invalid_pairs_count += 1
                print(f"빈 라벨 파일과 해당 이미지 파일 '{file_name}', '{image_file}'이 유효하지 않습니다.")
            else:
                # 유효한 데이터만 새로운 디렉토리에 복사
                valid_image_path = os.path.join(new_output_dir, file_name.replace('.txt', '.jpg'))
                valid_label_path = os.path.join(new_output_dir, file_name)

                # 이미지와 라벨 복사
                shutil.copyfile(os.path.join(output_dir, file_name.replace('.txt', '.jpg')), valid_image_path)
                shutil.copyfile(file_path, valid_label_path)

    print(f"총 {invalid_pairs_count}개의 쓸모없는 데이터 쌍이 발견되었습니다.")
    print(f"유효한 데이터는 '{new_output_dir}'에 복사되었습니다.")

# 빈 파일 및 해당 이미지 파일 수 확인 및 유효한 데이터 복사
filter_and_copy_valid_data(output_dir, new_output_dir)


In [ ]:
!pip uninstall -y imgaug
!pip install git+https://github.com/aleju/imgaug.git


In [ ]:
!pip install -U albumentations

In [ ]:
# import os
# import cv2
# import json
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import albumentations as A

# # 경로 설정
# label_dir = '/content/drive/MyDrive/yolo_training_A1/filtered_train'
# image_dir = '/content/drive/MyDrive/yolo_training_A1/filtered_train'
# output_dir = '/content/drive/MyDrive/yolo_training_A1/Augmented_Output_Data_A1'

# os.makedirs(output_dir, exist_ok=True)

# # 클래스 매핑
# augment_classes = [0, 3]  # 증강할 클래스 (0과 3)
# ignore_classes = [1, 2, 4, 5]  # 무시할 클래스

# # 데이터 증강을 위한 Albumentations 설정 (확대/축소 없이 이동만 적용)
# transform = A.Compose([
#     A.HorizontalFlip(p=0.5),  # 50% 확률로 수평으로 뒤집기
#     A.VerticalFlip(p=0.5),    # 50% 확률로 수직으로 뒤집기
#     A.RandomBrightnessContrast(p=0.2),  # 20% 확률로 밝기와 대비 조정
#     A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0, rotate_limit=0, p=0.5)  # 50% 확률로 이동만 적용
# ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# # 이미지와 라벨 파일을 처리하는 함수 (증강 포함)
# def augment_data(image_path, label_path, output_dir, aug_count=1):
#     # 이미지 읽기
#     image = cv2.imread(image_path)
#     if image is None:
#         print(f"Error: 이미지 '{image_path}'를 읽을 수 없습니다.")
#         return

#     # 라벨 읽기
#     label_name = os.path.basename(label_path)
#     image_name = os.path.basename(image_path)

#     # 증강된 이미지와 라벨의 이름에 접미사 추가
#     output_image_name = f"{os.path.splitext(image_name)[0]}_augmented{aug_count}.jpg"
#     output_label_name = f"{os.path.splitext(label_name)[0]}_augmented{aug_count}.txt"

#     output_image_path = os.path.join(output_dir, output_image_name)
#     output_label_path = os.path.join(output_dir, output_label_name)

#     with open(label_path, 'r') as f:
#         lines = f.readlines()

#     bboxes = []
#     class_labels = []

#     # 바운딩 박스와 클래스 라벨을 읽어옴
#     for line in lines:
#         class_id, x_center, y_center, width, height = map(float, line.strip().split())
#         class_id = int(class_id)

#         # 증강할 클래스만 처리 (0과 3)
#         if class_id in augment_classes:
#             bboxes.append([x_center, y_center, width, height])
#             class_labels.append(class_id)

#     # 증강 실행 (바운딩 박스가 있을 때만)
#     # if bboxes:
#     #     augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
#     #     augmented_image = augmented['image']
#     #     augmented_bboxes = augmented['bboxes']
#     #     augmented_classes = augmented['class_labels']
#     if bboxes:
#         print(f"증강 전 바운딩 박스: {bboxes}, 클래스 라벨: {class_labels}")  # 추가된 디버깅 메시지

#         # 클래스 라벨을 정수형으로 변환
#         class_labels = [int(label) for label in class_labels]

#         try:
#             augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
#             augmented_image = augmented['image']
#             augmented_bboxes = augmented['bboxes']
#             augmented_classes = augmented['class_labels']

#             # 증강된 이미지 저장
#             cv2.imwrite(output_image_path, augmented_image)

#             # 증강된 라벨 저장 (클래스 0과 3만 포함)
#             with open(output_label_path, 'w') as label_file:
#                 for class_id, bbox in zip(augmented_classes, augmented_bboxes):
#                     label_file.write(f"{class_id} {' '.join(map(str, bbox))}\n")
#         except KeyError as ke:
#             print(f"KeyError 발생: {ke}, 클래스 라벨: {class_labels}, 바운딩 박스: {bboxes}")
#             return
#         except Exception as e:
#             print(f"증강 중 다른 오류 발생: {type(e).__name__}, {e}")
#             return


# # 멀티스레드를 사용한 병렬 처리 함수
# def process_parallel():
#     with ThreadPoolExecutor() as executor:
#         futures = []
#         aug_count = 1  # 증강 횟수를 카운트할 변수

#         for label_file in os.listdir(label_dir):
#             if label_file.endswith('.txt'):  # 라벨 파일이 있는 경우
#                 image_file = label_file.replace('.txt', '.jpg')  # 동일한 이미지 파일명 찾기
#                 image_path = os.path.join(image_dir, image_file)
#                 label_path = os.path.join(label_dir, label_file)

#                 # 병렬 처리에서 증강 횟수를 추적하면서 전달
#                 futures.append(executor.submit(augment_data, image_path, label_path, output_dir, aug_count))
#                 aug_count += 1  # 증강할 때마다 카운트를 증가시킴

#         # 완료된 작업 확인
#         for future in as_completed(futures):
#             try:
#                 future.result()  # 결과를 받아서 예외를 처리할 수 있음
#             except Exception as e:
#                 print(f"Error 발생: {type(e).__name__}, {e}")

# # 병렬 처리 실행
# process_parallel()

# print("모든 데이터 증강이 완료되었습니다!")


In [ ]:
import os
import cv2
import uuid  # 고유한 파일명을 위해 UUID 추가
from concurrent.futures import ThreadPoolExecutor, as_completed
import albumentations as A

label_dir = '/content/drive/MyDrive/yolo_training_A1/filtered_train'
image_dir = '/content/drive/MyDrive/yolo_training_A1/filtered_train'
output_dir = '/content/drive/MyDrive/yolo_training_A1/Augmented_Output_Data_A1'
os.makedirs(output_dir, exist_ok=True)

augment_classes = {0: 3, 3: 10}  # 클래스별 목표 증강 횟수 설정
ignore_classes = [1, 2, 4, 5]  # 무시할 클래스
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0, rotate_limit=0, p=0.5)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# 증강 데이터 생성 함수
def augment_data(image_path, label_path, output_dir, class_id, target_count):
    image = cv2.imread(image_path)
    if image is None:
        print(f"[오류] 이미지 '{image_path}'를 읽을 수 없습니다.")
        return

    with open(label_path, 'r') as f:
        lines = f.readlines()

    # 증강할 클래스만 필터링
    bboxes = []
    class_labels = []
    for line in lines:
        line_class_id, x_center, y_center, width, height = map(float, line.strip().split())
        line_class_id = int(line_class_id)

        # 증강 대상 클래스만 추가
        if line_class_id == class_id:
            bboxes.append([x_center, y_center, width, height])
            class_labels.append(line_class_id)

    # 증강할 클래스가 포함된 경우에만 진행
    if bboxes:
        print(f"[진행중] '{image_path}'에서 클래스 {class_id} 증강 시작... (목표: {target_count}회)")
        for i in range(target_count):
            try:
                augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
                augmented_image = augmented['image']
                augmented_bboxes = augmented['bboxes']
                augmented_classes = augmented['class_labels']

                # 고유한 파일 이름 생성
                unique_id = uuid.uuid4().hex[:8]
                output_image_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(image_path))[0]}_{unique_id}.jpg")
                output_label_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(label_path))[0]}_{unique_id}.txt")

                # 증강된 이미지 저장
                cv2.imwrite(output_image_path, augmented_image)

                # 증강된 라벨 저장
                with open(output_label_path, 'w') as label_file:
                    for aug_class_id, bbox in zip(augmented_classes, augmented_bboxes):
                        label_file.write(f"{aug_class_id} {' '.join(map(str, bbox))}\n")

                print(f"[완료] '{output_image_path}' 및 라벨 저장 완료.")
            except KeyError as ke:
                print(f"[오류] KeyError 발생: {ke}, 클래스 라벨: {class_labels}, 바운딩 박스: {bboxes}")
            except Exception as e:
                print(f"[오류] 증강 중 다른 오류 발생: {type(e).__name__}, {e}")

# 멀티스레드 병렬 처리 함수
def process_parallel():
    with ThreadPoolExecutor() as executor:
        futures = []
        for label_file in os.listdir(label_dir):
            if label_file.endswith('.txt'):
                image_file = label_file.replace('.txt', '.jpg')
                image_path = os.path.join(image_dir, image_file)
                label_path = os.path.join(label_dir, label_file)

                # 라벨 파일 내용에서 증강 대상 클래스가 있는지 확인
                with open(label_path, 'r') as f:
                    lines = f.readlines()
                    has_augment_class = False
                    for line in lines:
                        line_class_id = int(line.strip().split()[0])
                        if line_class_id in augment_classes:
                            has_augment_class = True
                            break

                # 증강 대상 클래스가 포함된 경우만 증강 진행
                if has_augment_class:
                    print(f"[진행중] '{label_file}' 파일에서 증강 대상 클래스 확인됨.")
                    for class_id, target_count in augment_classes.items():
                        # 각 증강 대상 클래스별로 별도의 증강 수행
                        futures.append(executor.submit(augment_data, image_path, label_path, output_dir, class_id, target_count))

        for future in as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"[오류] Error 발생: {type(e).__name__}, {e}")

process_parallel()
print("모든 데이터 증강이 완료되었습니다!")


In [ ]:
import os

label_dir = '/content/drive/MyDrive/yolo_training_A1/train'

# 파일 목록을 출력
try:
    files = os.listdir(label_dir)
    print(files)
except OSError as e:
    print(f"Error accessing the directory: {e}")


In [ ]:
import os
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor

# YOLO 형식의 라벨이 있는 디렉토리 설정
label_dir = '/content/drive/MyDrive/yolo_training_A1/Merged_Data_A1'

# 클래스별 객체 수 저장 딕셔너리
class_counts = defaultdict(int)

# 라벨 파일에서 클래스 ID 읽어서 카운팅하는 함수
def process_label_file(label_file):
    label_path = os.path.join(label_dir, label_file)
    local_class_counts = defaultdict(int)

    if label_file.endswith('.txt'):
        with open(label_path, 'r') as f:
            for line in f:
                class_id = int(line.split()[0])  # 클래스 ID는 라벨 파일의 첫 번째 값
                local_class_counts[class_id] += 1
    return local_class_counts

# 병렬로 파일 처리
with ThreadPoolExecutor() as executor:
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

    # 멀티스레드로 파일 처리
    results = executor.map(process_label_file, label_files)

    # 결과 합산
    for local_class_counts in results:
        for class_id, count in local_class_counts.items():
            class_counts[class_id] += count

# 클래스별 개수 출력
for class_id, count in class_counts.items():
    print(f"클래스 ID {class_id}의 객체 수: {count}")


In [ ]:
import os
import shutil

# 원본 디렉토리 경로
augmented_dir = '/content/drive/MyDrive/yolo_training_A1/Augmented_Output_Data_A1'
filtered_dir = '/content/drive/MyDrive/yolo_training_A1/filtered_train'

# 새로운 병합 디렉토리 경로
merged_dir = '/content/drive/MyDrive/yolo_training_A1/Merged_Data_A1'
os.makedirs(merged_dir, exist_ok=True)

# 디렉토리에서 파일 복사하는 함수
def copy_files_to_merged_dir(src_dir, dst_dir):
    for file_name in os.listdir(src_dir):
        src_path = os.path.join(src_dir, file_name)
        dst_path = os.path.join(dst_dir, file_name)

        # 파일이 이미 존재하는지 확인하고, 없다면 복사
        if not os.path.exists(dst_path):
            shutil.copy2(src_path, dst_path)
            print(f"[복사 완료] '{src_path}' -> '{dst_path}'")
        else:
            print(f"[중복 파일] '{file_name}'은 이미 '{dst_dir}'에 존재하여 건너뜁니다.")

# 각 원본 디렉토리에서 파일을 병합 디렉토리로 복사
print("증강 데이터 파일 복사 중...")
copy_files_to_merged_dir(augmented_dir, merged_dir)

print("필터링된 데이터 파일 복사 중...")
copy_files_to_merged_dir(filtered_dir, merged_dir)

print("모든 파일이 병합 디렉토리에 복사되었습니다.")


In [ ]:
# # 두 개의 폴더 경로 설정
# LabeledData_path = '/content/drive/MyDrive/01-1.정식개방데이터/Output_LabeledData_A1'
# SourceData_path = '/content/drive/MyDrive/01-1.정식개방데이터/Output_SourceData_A1'

# # 폴더 내 파일 개수 세기
# LabeledData_count = len([name for name in os.listdir(LabeledData_path) if os.path.isfile(os.path.join(LabeledData_path, name))])
# SourceData_count = len([name for name in os.listdir(SourceData_path) if os.path.isfile(os.path.join(SourceData_path, name))])

# # 결과 출력
# print(f"라벨링데이터의 파일 개수: {LabeledData_count}")
# print(f"원천데이터의 파일 개수: {SourceData_count}")

In [ ]:
# import json


# # 이미지와 라벨 파일 경로 설정 (구글 드라이브 내 경로로 설정)

# image_dir = '/content/drive/MyDrive/01-1.정식개방데이터/Output_SourceData_A1'  # 이미지 저장 경로
# label_dir = '/content/drive/MyDrive/01-1.정식개방데이터/Output_LabeledData_A1'  # 라벨 파일 저장 경로
# # 이미지와 라벨 파일 리스트 얻기
# image_files = set([os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith('.jpg')])
# label_files = set([os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith('.txt')])

# # 이미지와 라벨 모두 있는 파일 리스트 찾기
# valid_files = image_files.intersection(label_files)

# # 이미지나 라벨만 있는 파일 리스트
# only_image_files = image_files - valid_files
# only_label_files = label_files - valid_files

# # 이미지나 라벨 중 하나만 있는 파일 삭제 (필요시)
# for file in only_image_files:
#     os.remove(os.path.join(image_dir, file + '.jpg'))

# for file in only_label_files:
#     os.remove(os.path.join(label_dir, file + '.txt'))

# print(f"총 {len(valid_files)}개의 유효한 이미지와 라벨 파일이 있습니다.")

In [ ]:
# 갯수 확인
import os

# 이건 하나의 폴더에 있을때 거기서 갯수 확인해보는거
# 두 개의 폴더가 아닌 하나의 폴더 경로 설정
data_dir = '/content/drive/MyDrive/yolo_training_A1/Merged_Data_A1'

# 이미지와 라벨 파일 구분하여 리스트 생성
image_files = [name for name in os.listdir(data_dir) if name.endswith('.jpg')]
label_files = [name for name in os.listdir(data_dir) if name.endswith('.txt')]

# 이미지 파일과 라벨 파일 개수 확인
image_count = len(image_files)
label_count = len(label_files)

# 결과 출력
print(f"이미지 파일 개수: {image_count}")
print(f"라벨 파일 개수: {label_count}")


In [ ]:
# 이미지와 라벨 매칭 확인

# 이것도 하나의 폴더에 있을때 손실난 부분 맞추는거
# 이미지와 라벨 파일 리스트 얻기 (같은 폴더 내에서 가져옴)
image_files = set([os.path.splitext(f)[0] for f in os.listdir(data_dir) if f.endswith('.jpg')])
label_files = set([os.path.splitext(f)[0] for f in os.listdir(data_dir) if f.endswith('.txt')])

# 이미지와 라벨 모두 있는 파일 리스트 찾기
valid_files = image_files.intersection(label_files)

# 이미지나 라벨만 있는 파일 리스트
only_image_files = image_files - valid_files
only_label_files = label_files - valid_files

# 이미지나 라벨 중 하나만 있는 파일 삭제 (필요시)
# for file in only_image_files:
#     os.remove(os.path.join(data_dir, file + '.jpg'))

# for file in only_label_files:
#     os.remove(os.path.join(data_dir, file + '.txt'))

print(f"총 {len(valid_files)}개의 유효한 이미지와 라벨 파일이 있습니다.")


In [ ]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor

# 원본 검증 데이터 경로 및 새로운 검증 데이터 경로 정의
original_val_dir = '/content/drive/MyDrive/yolo_training_A1/filtered_val'  # 원본 검증 데이터 경로
new_val_dir = '/content/drive/MyDrive/yolo_training_A1/augmented_val'  # 새로운 검증 데이터 경로

# 새로운 검증 데이터 폴더 생성
os.makedirs(new_val_dir, exist_ok=True)

# 검증 데이터에서 클래스 0, 3만 포함된 데이터를 추출하는 함수 정의
def process_file(filename):
    if filename.endswith('.jpg'):
        # 이미지 파일 경로 설정
        img_path = os.path.join(original_val_dir, filename)
        label_path = img_path.replace('.jpg', '.txt')

        # 라벨 파일이 존재하지 않는 경우 건너뜀
        if not os.path.exists(label_path):
            return

        # 라벨 파일에서 클래스 0, 3 정보 추출
        new_label_lines = []
        with open(label_path, 'r') as label_file:
            lines = label_file.readlines()
            for line in lines:
                parts = line.split()  # YOLO 형식의 각 줄을 공백으로 분할
                class_id = int(parts[0])  # 첫 번째 요소가 클래스 ID
                if class_id in [0, 3]:  # 클래스 0 또는 3만 남김
                    new_label_lines.append(line)

        # 클래스 0, 3만 포함된 데이터가 존재하는 경우에만 이미지와 수정된 라벨 파일을 복사
        if new_label_lines:
            # 이미지 파일 복사
            shutil.copy(img_path, os.path.join(new_val_dir, filename))

            # 수정된 라벨 파일 생성
            new_label_path = os.path.join(new_val_dir, os.path.basename(label_path))
            with open(new_label_path, 'w') as new_label_file:
                new_label_file.writelines(new_label_lines)

# 멀티스레딩을 통해 데이터 처리
with ThreadPoolExecutor(max_workers=8) as executor:  # 최대 8개의 스레드 사용
    executor.map(process_file, os.listdir(original_val_dir))


In [ ]:
# 갯수 확인
import os

# 이건 하나의 폴더에 있을때 거기서 갯수 확인해보는거
# 두 개의 폴더가 아닌 하나의 폴더 경로 설정
data_dir = '/content/drive/MyDrive/yolo_training_A1/augmented_val'

# 이미지와 라벨 파일 구분하여 리스트 생성
image_files = [name for name in os.listdir(data_dir) if name.endswith('.jpg')]
label_files = [name for name in os.listdir(data_dir) if name.endswith('.txt')]

# 이미지 파일과 라벨 파일 개수 확인
image_count = len(image_files)
label_count = len(label_files)

# 결과 출력
print(f"이미지 파일 개수: {image_count}")
print(f"라벨 파일 개수: {label_count}")

In [ ]:
# 이미지와 라벨 매칭 확인

# 이것도 하나의 폴더에 있을때 손실난 부분 맞추는거
# 이미지와 라벨 파일 리스트 얻기 (같은 폴더 내에서 가져옴)
image_files = set([os.path.splitext(f)[0] for f in os.listdir(data_dir) if f.endswith('.jpg')])
label_files = set([os.path.splitext(f)[0] for f in os.listdir(data_dir) if f.endswith('.txt')])

# 이미지와 라벨 모두 있는 파일 리스트 찾기
valid_files = image_files.intersection(label_files)

# 이미지나 라벨만 있는 파일 리스트
only_image_files = image_files - valid_files
only_label_files = label_files - valid_files

# 이미지나 라벨 중 하나만 있는 파일 삭제 (필요시)
# for file in only_image_files:
#     os.remove(os.path.join(data_dir, file + '.jpg'))

# for file in only_label_files:
#     os.remove(os.path.join(data_dir, file + '.txt'))

print(f"총 {len(valid_files)}개의 유효한 이미지와 라벨 파일이 있습니다.")

In [ ]:
import os
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor

# YOLO 형식의 라벨이 있는 디렉토리 설정
label_dir = '/content/drive/MyDrive/yolo_training_A1/augmented_val'

# 클래스별 객체 수 저장 딕셔너리
class_counts = defaultdict(int)

# 라벨 파일에서 클래스 ID 읽어서 카운팅하는 함수
def process_label_file(label_file):
    label_path = os.path.join(label_dir, label_file)
    local_class_counts = defaultdict(int)

    if label_file.endswith('.txt'):
        with open(label_path, 'r') as f:
            for line in f:
                class_id = int(line.split()[0])  # 클래스 ID는 라벨 파일의 첫 번째 값
                local_class_counts[class_id] += 1
    return local_class_counts

# 병렬로 파일 처리
with ThreadPoolExecutor() as executor:
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

    # 멀티스레드로 파일 처리
    results = executor.map(process_label_file, label_files)

    # 결과 합산
    for local_class_counts in results:
        for class_id, count in local_class_counts.items():
            class_counts[class_id] += count

# 클래스별 개수 출력
for class_id, count in class_counts.items():
    print(f"클래스 ID {class_id}의 객체 수: {count}")

In [ ]:

!cp /content/drive/MyDrive/yolo_training_A1/filtered_train-20241109T202844Z-001.zip /content/


In [ ]:
import zipfile
import concurrent.futures
import os

# 압축 해제할 경로 설정
extract_path = '/content/filtered_train_data'
os.makedirs(extract_path, exist_ok=True)

# ZIP 파일 목록
zip_files = ['/content/filtered_train-20241109T202844Z-001.zip']

# ZIP 파일을 압축 해제하는 함수 정의
def extract_zip(zip_file):
    try:
        sub_extract_path = os.path.join(extract_path, os.path.basename(zip_file).replace('.zip', ''))
        os.makedirs(sub_extract_path, exist_ok=True)
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f'{zip_file} 압축 해제 완료')
    except Exception as e:
        print(f'{zip_file} 압축 해제 중 오류 발생: {e}')

# 멀티스레딩을 사용하여 ZIP 파일을 병렬로 압축 해제
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(extract_zip, zip_files)

print("모든 ZIP 파일이 멀티스레드 방식으로 한 폴더에 압축 해제되었습니다.")


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from collections import Counter

# 원본 데이터 경로 및 새로운 데이터 경로 정의
original_dir = '/content/filtered_train_data'  # 학습 데이터 폴더
train_output_dir = '/content/drive/MyDrive/filtered_train_balanced'  # 학습 데이터 출력 폴더
val_output_dir = '/content/drive/MyDrive/filtered_val_balanced'  # 검증 데이터 출력 폴더

# 새로운 폴더 생성
os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(val_output_dir, exist_ok=True)

# 각 이미지의 클래스 레이블을 저장하기 위한 리스트 초기화
image_paths = []
class_labels = []

# 데이터셋에서 각 이미지에 포함된 클래스 정보를 확인하고 저장
for root, dirs, files in os.walk(original_dir):
    for filename in files:
        if filename.endswith('.txt'):
            label_path = os.path.join(root, filename)
            with open(label_path, 'r') as label_file:
                lines = label_file.readlines()
                classes_in_image = set()
                for line in lines:
                    class_id = int(line.split()[0])  # 클래스 ID 추출
                    classes_in_image.add(class_id)

                # 이미지 파일과 해당 클래스 레이블 정보 저장
                image_filename = filename.replace('.txt', '.jpg')
                image_path = os.path.join(root, image_filename)
                image_paths.append(image_path)

                # 클래스 조합을 문자열로 변환하여 고유한 레이블로 사용
                class_combination = '-'.join(map(str, sorted(classes_in_image)))
                class_labels.append(class_combination)

# 클래스 조합 빈도수 확인
class_counts = Counter(class_labels)

# 빈도수가 2개 미만인 클래스 조합 필터링 방지
filtered_image_paths = image_paths
filtered_class_labels = class_labels

# 학습 데이터와 검증 데이터 나누기 (8:2 비율), 클래스 비율을 유지
if len(filtered_class_labels) > 1:  # 클래스 레이블이 충분히 존재하는 경우에만 분할
    try:
        X_train, X_val, y_train, y_val = train_test_split(
            filtered_image_paths, filtered_class_labels, test_size=0.2, random_state=42, stratify=filtered_class_labels
        )
    except ValueError:
        # 클래스 레이블이 충분하지 않은 경우, 데이터를 임의로 나누되 데이터 손실을 방지
        split_index = int(len(filtered_image_paths) * 0.8)
        X_train, X_val = filtered_image_paths[:split_index], filtered_image_paths[split_index:]
        y_train, y_val = filtered_class_labels[:split_index], filtered_class_labels[split_index:]
else:
    raise ValueError("클래스 레이블이 충분하지 않습니다. 데이터셋을 확인하세요.")

# 멀티스레딩을 사용하여 파일 복사 작업 수행
def copy_files(img_filename, output_dir):
    img_path = img_filename  # img_filename은 이미 전체 경로를 포함하고 있음
    label_path = img_path.replace('.jpg', '.txt')

    # 이미지 파일 복사
    shutil.copy(img_path, os.path.join(output_dir, os.path.basename(img_filename)))
    # 라벨 파일 복사
    shutil.copy(label_path, os.path.join(output_dir, os.path.basename(label_path)))

# 멀티스레드 실행 설정 (최대 8개의 스레드 사용)
with ThreadPoolExecutor(max_workers=8) as executor:
    # 학습 데이터 복사
    executor.map(lambda img: copy_files(img, train_output_dir), X_train)
    # 검증 데이터 복사
    executor.map(lambda img: copy_files(img, val_output_dir), X_val)


In [ ]:
import os
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor

# YOLO 형식의 라벨이 있는 디렉토리 설정
label_dir = '/content/drive/MyDrive/01-1.정식개방데이터/Output_Data_00/A5/train/A5'

# 클래스별 객체 수 저장 딕셔너리
class_counts = defaultdict(int)

# 라벨 파일에서 클래스 ID 읽어서 카운팅하는 함수
def process_label_file(label_file):
    label_path = os.path.join(label_dir, label_file)
    local_class_counts = defaultdict(int)

    if label_file.endswith('.txt'):
        with open(label_path, 'r') as f:
            for line in f:
                class_id = int(line.split()[0])  # 클래스 ID는 라벨 파일의 첫 번째 값
                local_class_counts[class_id] += 1
    return local_class_counts

# 병렬로 파일 처리
with ThreadPoolExecutor() as executor:
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

    # 멀티스레드로 파일 처리
    results = executor.map(process_label_file, label_files)

    # 결과 합산
    for local_class_counts in results:
        for class_id, count in local_class_counts.items():
            class_counts[class_id] += count

# 클래스별 개수 출력
for class_id, count in class_counts.items():
    print(f"클래스 ID {class_id}의 객체 수: {count}")

In [ ]:
# 갯수 확인
import os

# 이건 하나의 폴더에 있을때 거기서 갯수 확인해보는거
# 두 개의 폴더가 아닌 하나의 폴더 경로 설정
data_dir = '/content/drive/MyDrive/01-1.정식개방데이터/Output_Data_00/A5/train/A5'

# 이미지와 라벨 파일 구분하여 리스트 생성
image_files = [name for name in os.listdir(data_dir) if name.endswith('.jpg')]
label_files = [name for name in os.listdir(data_dir) if name.endswith('.txt')]

# 이미지 파일과 라벨 파일 개수 확인
image_count = len(image_files)
label_count = len(label_files)

# 결과 출력
print(f"이미지 파일 개수: {image_count}")
print(f"라벨 파일 개수: {label_count}")

In [ ]:
# 첫 번째 폴더 압축
!zip -r /content/filtered_train_balanced.zip /content/drive/MyDrive/filtered_train_balanced

# 두 번째 폴더 압축
!zip -r /content/filtered_val_balanced.zip /content/drive/MyDrive/filtered_val_balanced

In [ ]:
import zipfile
import concurrent.futures
import os

# 압축 해제할 경로 설정
extract_path = '/content/filtered_train_balanced'
os.makedirs(extract_path, exist_ok=True)

# ZIP 파일 목록
zip_files = ['/content/filtered_train_balanced.zip']

# ZIP 파일을 압축 해제하는 함수 정의
def extract_zip(zip_file):
    try:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f'{zip_file} 압축 해제 완료')
    except Exception as e:
        print(f'{zip_file} 압축 해제 중 오류 발생: {e}')

# 멀티스레딩을 사용하여 ZIP 파일을 병렬로 압축 해제
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(extract_zip, zip_files)

print("모든 ZIP 파일이 멀티스레드 방식으로 한 폴더에 압축 해제되었습니다.")


In [ ]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 원본 데이터셋 경로와 새 폴더 경로 설정
original_dataset_path = '/content/filtered_train_balanced/content/drive/MyDrive/filtered_train_balanced'  # 이미지와 라벨 데이터가 같이 있는 폴더
new_dataset_path = '/content/new_train_A1'  # 클래스 3을 포함하는 데이터가 저장될 폴더

# 새 폴더 생성
os.makedirs(new_dataset_path, exist_ok=True)

# 라벨 데이터가 있는 경로 설정
label_files_path = [file for file in os.listdir(original_dataset_path) if file.endswith('.txt')]

# 클래스 3을 포함하는 데이터 필터링 함수 정의
def filter_and_copy(label_file):
    label_file_path = os.path.join(original_dataset_path, label_file)
    with open(label_file_path, 'r') as f:
        lines = f.readlines()

    # 클래스 3을 포함하는지 확인
    contains_class_3 = any(line.split()[0] == '3' for line in lines)

    if contains_class_3:
        # 해당 라벨 파일을 새로운 폴더로 복사
        shutil.copy(label_file_path, os.path.join(new_dataset_path, label_file))

        # 동일한 이름의 이미지 파일도 복사
        image_file = label_file.replace('.txt', '.jpg')
        image_file_path = os.path.join(original_dataset_path, image_file)
        if os.path.exists(image_file_path):
            shutil.copy(image_file_path, os.path.join(new_dataset_path, image_file))

# 멀티스레드 방식으로 클래스 3을 포함하는 데이터 필터링 및 복사
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(filter_and_copy, label_files_path), total=len(label_files_path), desc="Filtering labels containing class 3"))

print("데이터 필터링 완료: 클래스 3을 포함하는 데이터들만 새 폴더에 복사되었습니다.")


In [ ]:
import os
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor

# YOLO 형식의 라벨이 있는 디렉토리 설정
label_dir = '/content/new_train_A1'

# 클래스별 객체 수 저장 딕셔너리
class_counts = defaultdict(int)

# 라벨 파일에서 클래스 ID 읽어서 카운팅하는 함수
def process_label_file(label_file):
    label_path = os.path.join(label_dir, label_file)
    local_class_counts = defaultdict(int)

    if label_file.endswith('.txt'):
        with open(label_path, 'r') as f:
            for line in f:
                class_id = int(line.split()[0])  # 클래스 ID는 라벨 파일의 첫 번째 값
                local_class_counts[class_id] += 1
    return local_class_counts

# 병렬로 파일 처리
with ThreadPoolExecutor() as executor:
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

    # 멀티스레드로 파일 처리
    results = executor.map(process_label_file, label_files)

    # 결과 합산
    for local_class_counts in results:
        for class_id, count in local_class_counts.items():
            class_counts[class_id] += count

# 클래스별 개수 출력
for class_id, count in class_counts.items():
    print(f"클래스 ID {class_id}의 객체 수: {count}")

In [ ]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 원본 데이터셋 경로와 새 폴더 경로 설정
original_dataset_path = '/content/filtered_train_balanced/content/drive/MyDrive/filtered_train_balanced'  # 이미지와 라벨 데이터가 같이 있는 폴더
new_dataset_path = '/content/new_train_A1_with_0'  # 클래스 3을 포함하는 데이터가 저장될 폴더

# 새 폴더 생성
os.makedirs(new_dataset_path, exist_ok=True)

# 라벨 데이터가 있는 경로 설정
label_files_path = [file for file in os.listdir(original_dataset_path) if file.endswith('.txt')]

# 클래스 3을 포함하는 데이터 필터링 함수 정의
def filter_and_copy(label_file):
    label_file_path = os.path.join(original_dataset_path, label_file)
    with open(label_file_path, 'r') as f:
        lines = f.readlines()

    # 클래스 3을 포함하는지 확인
    contains_class_3 = any(line.split()[0] == '0' for line in lines)

    if contains_class_3:
        # 해당 라벨 파일을 새로운 폴더로 복사
        shutil.copy(label_file_path, os.path.join(new_dataset_path, label_file))

        # 동일한 이름의 이미지 파일도 복사
        image_file = label_file.replace('.txt', '.jpg')
        image_file_path = os.path.join(original_dataset_path, image_file)
        if os.path.exists(image_file_path):
            shutil.copy(image_file_path, os.path.join(new_dataset_path, image_file))

# 멀티스레드 방식으로 클래스 3을 포함하는 데이터 필터링 및 복사
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(filter_and_copy, label_files_path), total=len(label_files_path), desc="Filtering labels containing class 3"))

print("데이터 필터링 완료: 클래스 3을 포함하는 데이터들만 새 폴더에 복사되었습니다.")


In [ ]:
import os
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor

# YOLO 형식의 라벨이 있는 디렉토리 설정
label_dir = '/content/new_train_A1_with_0'

# 클래스별 객체 수 저장 딕셔너리
class_counts = defaultdict(int)

# 라벨 파일에서 클래스 ID 읽어서 카운팅하는 함수
def process_label_file(label_file):
    label_path = os.path.join(label_dir, label_file)
    local_class_counts = defaultdict(int)

    if label_file.endswith('.txt'):
        with open(label_path, 'r') as f:
            for line in f:
                class_id = int(line.split()[0])  # 클래스 ID는 라벨 파일의 첫 번째 값
                local_class_counts[class_id] += 1
    return local_class_counts

# 병렬로 파일 처리
with ThreadPoolExecutor() as executor:
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

    # 멀티스레드로 파일 처리
    results = executor.map(process_label_file, label_files)

    # 결과 합산
    for local_class_counts in results:
        for class_id, count in local_class_counts.items():
            class_counts[class_id] += count

# 클래스별 개수 출력
for class_id, count in class_counts.items():
    print(f"클래스 ID {class_id}의 객체 수: {count}")

In [ ]:
import zipfile
import concurrent.futures
import os

# 압축 해제할 경로 설정
extract_path = '/content/filtered_val_balanced'
os.makedirs(extract_path, exist_ok=True)

# ZIP 파일 목록
zip_files = ['/content/filtered_val_balanced.zip']

# ZIP 파일을 압축 해제하는 함수 정의
def extract_zip(zip_file):
    try:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f'{zip_file} 압축 해제 완료')
    except Exception as e:
        print(f'{zip_file} 압축 해제 중 오류 발생: {e}')

# 멀티스레딩을 사용하여 ZIP 파일을 병렬로 압축 해제
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(extract_zip, zip_files)

print("모든 ZIP 파일이 멀티스레드 방식으로 한 폴더에 압축 해제되었습니다.")


In [ ]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 원본 데이터셋 경로와 새 폴더 경로 설정
original_dataset_path = '/content/filtered_val_balanced/content/drive/MyDrive/filtered_val_balanced'  # 이미지와 라벨 데이터가 같이 있는 폴더
new_dataset_path = '/content/new_val_A1'  # 클래스 3을 포함하는 데이터가 저장될 폴더

# 새 폴더 생성
os.makedirs(new_dataset_path, exist_ok=True)

# 라벨 데이터가 있는 경로 설정
label_files_path = [file for file in os.listdir(original_dataset_path) if file.endswith('.txt')]

# 클래스 3을 포함하는 데이터 필터링 함수 정의
def filter_and_copy(label_file):
    label_file_path = os.path.join(original_dataset_path, label_file)
    with open(label_file_path, 'r') as f:
        lines = f.readlines()

    # 클래스 3을 포함하는지 확인
    contains_class_3 = any(line.split()[0] == '3' for line in lines)

    if contains_class_3:
        # 해당 라벨 파일을 새로운 폴더로 복사
        shutil.copy(label_file_path, os.path.join(new_dataset_path, label_file))

        # 동일한 이름의 이미지 파일도 복사
        image_file = label_file.replace('.txt', '.jpg')
        image_file_path = os.path.join(original_dataset_path, image_file)
        if os.path.exists(image_file_path):
            shutil.copy(image_file_path, os.path.join(new_dataset_path, image_file))

# 멀티스레드 방식으로 클래스 3을 포함하는 데이터 필터링 및 복사
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(filter_and_copy, label_files_path), total=len(label_files_path), desc="Filtering labels containing class 3"))

print("데이터 필터링 완료: 클래스 3을 포함하는 데이터들만 새 폴더에 복사되었습니다.")


In [ ]:
import os
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor

# YOLO 형식의 라벨이 있는 디렉토리 설정
label_dir = '/content/new_val_A1'

# 클래스별 객체 수 저장 딕셔너리
class_counts = defaultdict(int)

# 라벨 파일에서 클래스 ID 읽어서 카운팅하는 함수
def process_label_file(label_file):
    label_path = os.path.join(label_dir, label_file)
    local_class_counts = defaultdict(int)

    if label_file.endswith('.txt'):
        with open(label_path, 'r') as f:
            for line in f:
                class_id = int(line.split()[0])  # 클래스 ID는 라벨 파일의 첫 번째 값
                local_class_counts[class_id] += 1
    return local_class_counts

# 병렬로 파일 처리
with ThreadPoolExecutor() as executor:
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

    # 멀티스레드로 파일 처리
    results = executor.map(process_label_file, label_files)

    # 결과 합산
    for local_class_counts in results:
        for class_id, count in local_class_counts.items():
            class_counts[class_id] += count

# 클래스별 개수 출력
for class_id, count in class_counts.items():
    print(f"클래스 ID {class_id}의 객체 수: {count}")

In [ ]:
# 갯수 확인
import os

# 이건 하나의 폴더에 있을때 거기서 갯수 확인해보는거
# 두 개의 폴더가 아닌 하나의 폴더 경로 설정
data_dir = '/content/new_val_A1'

# 이미지와 라벨 파일 구분하여 리스트 생성
image_files = [name for name in os.listdir(data_dir) if name.endswith('.jpg')]
label_files = [name for name in os.listdir(data_dir) if name.endswith('.txt')]

# 이미지 파일과 라벨 파일 개수 확인
image_count = len(image_files)
label_count = len(label_files)

# 결과 출력
print(f"이미지 파일 개수: {image_count}")
print(f"라벨 파일 개수: {label_count}")

In [ ]:
# 갯수 확인
import os

# 이건 하나의 폴더에 있을때 거기서 갯수 확인해보는거
# 두 개의 폴더가 아닌 하나의 폴더 경로 설정
data_dir = '/content/new_train_A1'

# 이미지와 라벨 파일 구분하여 리스트 생성
image_files = [name for name in os.listdir(data_dir) if name.endswith('.jpg')]
label_files = [name for name in os.listdir(data_dir) if name.endswith('.txt')]

# 이미지 파일과 라벨 파일 개수 확인
image_count = len(image_files)
label_count = len(label_files)

# 결과 출력
print(f"이미지 파일 개수: {image_count}")
print(f"라벨 파일 개수: {label_count}")

In [ ]:
!zip -r new_val.zip /content/new_val_A1
!zip -r new_train.zip /content/new_train_A1
